# Imports

In [2]:
import cv2
import numpy as np
import pandas as pd
import time
import pickle
import PIL
import os
import winsound

from matplotlib import pyplot as plt

from pykinect2 import PyKinectV2
from pykinect2.PyKinectV2 import *
from pykinect2 import PyKinectRuntime

from collections import deque

# Recording data from Kinect v2

In [29]:
def play_beep(duration):
    winsound.Beep(2500, duration)


In [30]:
kinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Color | PyKinectV2.FrameSourceTypes_Depth)
kinect.close()

In [31]:
def reshape_frames(color_frames, depth_frames):
    reshaped_color_frames = deque(maxlen=60)
    reshaped_depth_frames = deque(maxlen=60)

    for color_frame in color_frames:
        reshaped_color_frame = color_frame.reshape((kinect.color_frame_desc.Height, kinect.color_frame_desc.Width, -1)).astype(np.uint8)
        reshaped_color_frame = cv2.resize(reshaped_color_frame, (681,383))
        reshaped_color_frame = reshaped_color_frame[:,158:541,:]
        print("SHAPE", reshaped_color_frame.shape)
        reshaped_color_frames.append(reshaped_color_frame)

    for depth_frame in depth_frames:
        reshaped_depth_frame = depth_frame.reshape((kinect.depth_frame_desc.Height, kinect.depth_frame_desc.Width)).astype(np.uint16)
        reshaped_depth_frame = reshaped_depth_frame[16:399,64:447]
        reshaped_depth_frames.append(reshaped_depth_frame)

    return reshaped_color_frames, reshaped_depth_frames

def save_to_npz(name, action, color_frames, depth_frames):
    color_file_path = f'./numpy_data_arrays/{action}/{name}_color.npz'
    depth_file_path = f'./numpy_data_arrays/{action}/{name}_depth.npz'
    np.savez_compressed(color_file_path, np.array(color_frames))
    np.savez_compressed(depth_file_path, np.array(depth_frames))

def record_to_video_files(name, action, color_frames, depth_frames):
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    color_file_path = f'./videos/{action}/{name}_color.mov'
    depth_file_path = f'./videos/{action}/{name}_depth.mov'

    color_video_writer = cv2.VideoWriter(color_file_path, fourcc, len(color_frames) // 2, (383, 383))
    depth_video_writer = cv2.VideoWriter(depth_file_path, fourcc, len(depth_frames) // 2, (383, 383), isColor=False)

    for color_frame in color_frames:
        frame_to_add = cv2.cvtColor(color_frame, cv2.COLOR_RGBA2RGB)
        color_video_writer.write(frame_to_add)
    color_video_writer.release()

    for depth_frame in depth_frames:
        frame_to_add = depth_frame.astype(np.uint8)

        depth_video_writer.write(frame_to_add)

    depth_video_writer.release()

    return None

def record_new_rgbd_data(name, action):
    color_frames = deque(maxlen=60)  
    depth_frames = deque(maxlen=60)
    timeStart = time.time()
    # WARM START / Countdown - 3 seconds of color and depth
    while time.time() < timeStart + 3:
        if kinect.has_new_color_frame():
            color_frame = kinect.get_last_color_frame()
            depth_frame = kinect.get_last_depth_frame()
    play_beep(200)
    # Record 2 seconds of color and depth
    while time.time() < timeStart + 5:
        if kinect.has_new_color_frame():
            color_frame = kinect.get_last_color_frame()
            depth_frame = kinect.get_last_depth_frame()
            color_frames.append(color_frame)
            depth_frames.append(depth_frame)
    play_beep(600)
    
    kinect.close()
    print(len(depth_frames))
    print("RECORDING DONE")
    reshaped_color_frames, reshaped_depth_frames = reshape_frames(color_frames, depth_frames)
    print("FRAMES RESHAPED")
    save_to_npz(name, action, reshaped_color_frames, reshaped_depth_frames)
    print("SAVED TO NPZ")
    record_to_video_files(name, action, reshaped_color_frames, reshaped_depth_frames)
    print("VIDEO RECORDED")

    return None

In [32]:
kinect = PyKinectRuntime.PyKinectRuntime(PyKinectV2.FrameSourceTypes_Color | PyKinectV2.FrameSourceTypes_Depth)
for i in range(1, 26):
    record_new_rgbd_data(f'jab_{i}', 'jab')
for i in range(1, 26):
    record_new_rgbd_data(f'cross_{i}', 'cross')
for i in range(1, 26):
    record_new_rgbd_data(f'left_hook_{i}', 'left_hook')
for i in range(1, 26):
    record_new_rgbd_data(f'right_hook_{i}', 'right_hook')
kinect.close()

TypeError: play_beep() missing 1 required positional argument: 'duration'